In [1]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib as plt

In [2]:
env = gym.make('MountainCar-v0')
def compute_modified_reward(next_state):
    modified_reward = np.square(max(0, next_state[0] + 0.5))
    if next_state[0] >= 0.5: 
        modified_reward += 1.0
    return modified_reward

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/jamal/AI_assign/lib/python3.5/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
input_state_size = 2
actions_size = 3
net_hidden_state = 32
batch_size = 128
lr = 0.01
max_iters = 500
max_t = 200


In [4]:
class Net:
    def __init__(self,input_state_size = input_state_size,
                 actions_size = actions_size, 
                 net_hidden_state = net_hidden_state,
                 lr = lr,
                 name='Net'):
        with tf.variable_scope(name):
            
            self.states = tf.placeholder(tf.float32,[None,input_state_size])
            self.actions = tf.placeholder(tf.int32,[None])
            self.hidden_layer = tf.contrib.layers.fully_connected(
                self.states, 
                net_hidden_state)
            self.hidden_layer = tf.contrib.layers.fully_connected(
                self.hidden_layer, 
                net_hidden_state)
            self.out = tf.contrib.layers.fully_connected(
                self.hidden_layer, 
                actions_size,
                activation_fn = None)
            
            self.one_hot_actions = tf.one_hot(
                self.actions, 
                actions_size)
            
            self.probabilities = tf.nn.softmax(self.out)
            
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.out,
                                                                                labels=self.one_hot_actions))
            self.optimizer = tf.train.AdamOptimizer(lr).minimize(self.loss)
    
    def get_action(self,state):
        feed_dict = {self.states : np.array([state])}
        
        probs = sess.run(self.probabilities,feed_dict = feed_dict)
        
        return np.random.choice(actions_size,p=probs[0])
        #$return np.argmax(probs[0])
    
    
    def train(self,batch):
        
        states , actions = zip(*batch)
        
        states = np.array(states)
        actions = np.array(actions)
        
        feed_dict = {self.states : states, self.actions : actions}
        
        _, pro = sess.run([self.optimizer,self.probabilities],feed_dict = feed_dict)        
        
        
        #print(pro)
        
            

In [5]:
tf.reset_default_graph()
net = Net(name = 'Net',net_hidden_state= net_hidden_state,lr=lr)

In [6]:
import random
import bisect
import time

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_index = int(max_iters * 80 / 100)
    
    while True:
        
        total_reward_list = []
        trajectory_list = []


        for i in np.arange(max_iters):
            total_reward = 0.0
            trajectory = []
            state = env.reset()

            for s in np.arange(max_t):
                action = net.get_action(state)

                next_state,reward,done,_ = env.step(action)
                #print(reward)
                modified_reward = compute_modified_reward(next_state)
                total_reward += modified_reward
                trajectory.append((state,action))

                state = next_state

                if done:
                    break

            index = bisect.bisect(total_reward_list,total_reward)
            total_reward_list.insert(index,total_reward)
            trajectory_list.insert(index,trajectory)
        #import pdb;pdb.set_trace()

        state_action_pairs = []

        for trajectory in trajectory_list[start_index:]:
            for state_action_pair in trajectory:
                state_action_pairs.append(state_action_pair)


        random.shuffle(state_action_pairs)

        batches = [state_action_pairs[k:k+batch_size] for k in np.arange(0,len(state_action_pairs),batch_size)]

        for batch in batches:
            net.train(batch)

        print("Now Testing")
        #test

        state = env.reset()
        env.render()
        time.sleep(0.01)
        total_reward = 0.0
        for s in np.arange(max_t):
            action = net.get_action(state)
            state,reward,done,_ = env.step(action)
            total_reward += reward
            env.render()
            time.sleep(0.01)
            if done:
                break


        env.close()
        print("Total reward:",total_reward)

        if total_reward > -110:
            break
        

Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -178.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -180.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -172.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -184.0
Now Testing
Total reward: -176.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -183.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -185.0
Now Testing
Total reward: -166.0
Now Testin

Total reward: -151.0
Now Testing
Total reward: -158.0
Now Testing
Total reward: -151.0
Now Testing
Total reward: -153.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -154.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -157.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -154.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -160.0
Now Testing
Total reward: -155.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -156.0
Now Testing
Total reward: -158.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -161.0
Now Testing
Total reward: -155.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -200.0
Now Testing
Total reward: -113.0
Now Testing
Total reward: -110.0
Now Testing
Total reward: -145.0
Now Testing
Total reward: -194.0
Now Testing
Total rewa